# Event Study in Python
- This program contains the codes that can be used in drawing the event study graph.
- The dataset, *Crsp_2010_2020.csv*, includes the return data for firms between 2010 and 2020. 
- The dataset, *Event.csv*, provides the template for the event input. Permno is a firm identifier for stock return data. Event_date should be formated as "Month/Day/Year".
- **Your task** is to understand the coding procedures and then answer the question listed below (Questions 1-4).

### Part A: Clean Data

**Loading pandas and dataset**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt #Package for drawing pictures
pd.options.mode.chained_assignment = None #Set off some pandas outputs

In [ ]:
#Import event data
event = pd.read_csv('Events.csv',parse_dates = ['event_date'])
event

In [ ]:
#Duplicates check
event=event.drop_duplicates()
event

#### Question 1: How many unique events do we have in Event?

Your Answers:

In [ ]:
#Import stock price data
stockprice = pd.read_csv('Crsp_2010_2020.csv',parse_dates = ['date','calendar_day'])

- permno: security identifier
- date: trading calendar day
- calendar_day: trading calendar day
- trading_day: trading day index
- abnormal_ret: raw return adjusted for delisting return and market return

**Preprocess dataset before merging**

In [ ]:
#Filter Event Firms' Stock Price
relatedsp=pd.merge(event, stockprice, how = 'left', on = 'permno')[['permno', 'date', 'abnormal_ret','calendar_day','trading_day']]

In [ ]:
#Duplicate Repetitive Merge
relatedsp=relatedsp.drop_duplicates()

In [ ]:
#Reset Index for Loop
event=event.reset_index(drop=True)
relatedsp=relatedsp.reset_index(drop=True)

**Merge each event to the closest trading day**

In [ ]:
#This code is to loop over each event and match each event with the cloest trading day within [0,+7].
event_trade_date = pd.DataFrame()
for r in range(0,len(event)):
    #Select event related price
    relatedsp_temp=relatedsp[relatedsp['permno']==event.loc[r,"permno"]] 
    #Assign event date to a seperate variable
    relatedsp_temp['event_date']=event.loc[r,'event_date'] 
    #Use the calendar_day minus event day
    relatedsp_temp['relative_day']=(relatedsp_temp['calendar_day']-relatedsp_temp['event_date']).dt.days 
    #Select match within [0,+7]
    relatedsp_temp=relatedsp_temp[(relatedsp_temp['relative_day']>=0)&(relatedsp_temp['relative_day']<=7)]
    #Sort and filter the trading day with minimum day difference
    relatedsp_temp=relatedsp_temp.sort_values(by = 'relative_day', ascending = True)
    temp=relatedsp_temp.head(1)[['permno', 'event_date', 'trading_day']]
    event_trade_date = event_trade_date.append(temp.head(1), ignore_index = True)
    
print("Number of Matched with CRSP:",len(event_trade_date))
event_trade_date.rename(columns={'trading_day':'event_trading_day'}, inplace=True)
print(event_trade_date)

#### Question 2: How many unique events are in event_trade_date? Why is it smaller than your answers in Question 1?

Your Answers:

**Collect event window [-5,10] trading data**

In [ ]:
#This code is to assign the 16 trading days around the event date, and transform each trading day to an event day.

#Merge the event with event_trading_day to price dataset 
event_price=pd.merge(event_trade_date, relatedsp, how = 'left', on = ['permno'])
#Calculate the trading day difference
event_price['event_day']=event_price['trading_day']-event_price['event_trading_day']
#Filter by graph window [-5,+10]
event_price=event_price[(event_price['event_day']<=10) & (event_price['event_day']>=-5)]
#Sort variable
event_price=event_price.sort_values(by = ['permno','event_date','event_day'], ascending = True)
event_price=event_price[['permno', 'event_date', 'calendar_day','trading_day','event_day','abnormal_ret']]
event_price.head(16)

**Delete events without sufficient trading data**

In [ ]:
event_price = event_price[event_price.groupby(['permno','event_date']).event_day.transform(len) >=14]
event_price.shape

#### Question 3: Could you explain what does "event_price.groupby(['permno','event_date']).relative_trading_day.transform(len) >=14" do?

Your Answers:

**Generate cumulative return**

In [ ]:
event_price['car'] = event_price.groupby(['permno','event_date'])['abnormal_ret'].apply(lambda x: x.cumsum())
event_price.head(16)

### Part B: Event Study Graph

**Draw event study graph**

In [ ]:
mean_ar=event_price.groupby('event_day')['car'].mean()
plt.plot(mean_ar)

#### Question 4: How do you interpret the graph?

Your Answers: